In [1]:
from google import genai
from google.genai import types
import base64

import PIL.Image
from pillow_heif import register_heif_opener
register_heif_opener()


wardrobe = []

In [2]:
def get_text_description_and_attributes(image):
    # image = PIL.Image.open(img_location)
    
    prompt = [
        """Analyze this clothing item and provide:
        1. A detailed description (2-3 sentences)
        2. Structured attributes in JSON format with these fields:
           - main_category (MUST be one of: 'Tops', 'Bottoms', 'Outerwear')
           - primary_color
           - secondary_colors (list)
           - pattern (e.g., solid, striped, graphic, etc.)
           - style (e.g., casual, sporty, formal)
           - occasion_tags (list of suitable occasions)
           - formality_score (0-10, where 0 is very casual and 10 is very formal)
           
        Format your response as:
        DESCRIPTION: [detailed description]
        ATTRIBUTES: [JSON data]""",
        image
    ]
    

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text

    import re

    desc_match = re.search(r'DESCRIPTION:(.*?)ATTRIBUTES:', response_text, re.DOTALL)
    description = desc_match.group(1).strip() if desc_match else ""
    
    json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
        try:
            import json
            attributes = json.loads(json_str)
        except json.JSONDecodeError:
            attributes = {}
    else:
        attributes = {}
    
    return {
        "description": description,
        "attributes": attributes,
        "raw_response": response_text 
    }

In [5]:
response = get_text_description_and_attributes('./closet/IMG_0622.jpeg')

In [16]:
import json
import pprint

print("\n===== CLOTHING ANALYSIS RESULTS =====")
print(f"\nDescription:\n{response['description']}")
print("\nAttributes:")
pprint.pprint(response['attributes'], indent=2)


===== CLOTHING ANALYSIS RESULTS =====

Description:
The image depicts a dark olive green puffer jacket with a quilted design. It appears to have a high collar and a zip-up front, suggesting warmth and weather resistance.

Attributes:
{ 'formality_score': 3,
  'main_category': 'Outerwear',
  'occasion_tags': ['casual', 'outdoor', 'travel'],
  'pattern': 'solid',
  'primary_color': 'green',
  'secondary_colors': [],
  'style': 'casual'}


In [7]:
def preprocess_for_supabase(image_url, description_and_attributes):

    description = description_and_attributes.get("description", "")
    attributes = description_and_attributes.get("attributes", {})
    raw_response = description_and_attributes.get("raw_response", "")
    
    valid_categories = ['Tops', 'Bottoms', 'Outerwear']
    main_category = attributes.get('main_category')
    
    if not main_category or main_category not in valid_categories:
        main_category = 'Tops'
    

    secondary_colors = attributes.get('secondary_colors', [])
    if isinstance(secondary_colors, str):
        secondary_colors = [color.strip() for color in secondary_colors.split(',')]
    
    occasion_tags = attributes.get('occasion_tags', [])
    if isinstance(occasion_tags, str):
        occasion_tags = [tag.strip() for tag in occasion_tags.split(',')]
    
    formality_score = attributes.get('formality_score')
    if isinstance(formality_score, str):
        try:
            formality_score = int(float(formality_score))
        except (ValueError, TypeError):
            formality_score = 5 
    
    if not isinstance(formality_score, (int, float)) or formality_score < 0 or formality_score > 10:
        formality_score = 5 
    
    supabase_object = {
        'description': description,
        'main_category': main_category,
        'primary_color': attributes.get('primary_color', ''),
        'secondary_colors': secondary_colors,
        'pattern': attributes.get('pattern', ''),
        'style': attributes.get('style', ''),
        'occasion_tags': occasion_tags,
        'formality_score': formality_score,
        'image_url': image_url,
        'raw_ai_response': raw_response
    }
    
    return supabase_object

In [ ]:
def upload_to_supabase(img_loc):

    image = PIL.Image.open(img_loc)
    description_and_attributes = get_text_description_and_attributes(image)
    # supabase = create_client(supabase_url, supabase_key)
    
    img_buffer = io.BytesIO()
    image.save(img_buffer, format=image.format if image.format else "JPEG")
    img_bytes = img_buffer.getvalue()
    

    import uuid
    image_filename = f"{uuid.uuid4()}.jpg"
    
    try:
        storage_response = supabase.storage.from_(bucket_name).upload(
            image_filename,
            img_bytes,
            {"content-type": "image/jpeg"}
        )

        image_url = supabase.storage.from_(bucket_name).get_public_url(image_filename)

        supabase_object = preprocess_for_supabase(image_url, description_and_attributes)
        
        db_response = supabase.table('Cloths').insert(supabase_object).execute()
        
        return {
            "status": "success",
            "data": db_response.data,
            "supabase_object": supabase_object  
        }
        
    except Exception as e:
        return {
            "status": "error",
            "message": str(e)
        }

In [15]:
temp = preprocess_for_supabase('./closet/IMG_0622.jpeg',response)

In [13]:
type(temp)

dict

In [17]:
temp

{'description': "This appears to be a women's light-colored blouse, possibly made of a flowy fabric like silk or rayon. It features a loose, relaxed fit with possible button detailing on the front, suggesting it could be a tunic or a button-down shirt that is unbuttoned.",
 'main_category': 'Tops',
 'primary_color': 'Beige',
 'secondary_colors': ['White', 'Cream'],
 'pattern': 'Solid',
 'style': 'Casual',
 'occasion_tags': ['Casual', 'Work', 'Brunch', 'Everyday'],
 'formality_score': 4,
 'image_url': './closet/IMG_0622.jpeg',
 'raw_ai_response': 'Okay, I\'m ready to analyze the image. I will provide a description and JSON attributes based on what I see.\n\nDESCRIPTION: This appears to be a women\'s light-colored blouse, possibly made of a flowy fabric like silk or rayon. It features a loose, relaxed fit with possible button detailing on the front, suggesting it could be a tunic or a button-down shirt that is unbuttoned.\n\nATTRIBUTES:\n```json\n{\n  "main_category": "Tops",\n  "primary

In [ ]:
def pil_to_inline_data(image):
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    base64_img = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return {
        "inline_data": {
            "mime_type": "image/jpeg",
            "data": base64_img
        }
    }


def recommend_outfit_multimodal(shirts, pants, outerwear, context):
    def format_item_text(item):
        item_id = item.get("id", "unknown")
        description = item.get("description", "No description")
        attributes = item.get("attributes", {})

        color = attributes.get("primary_color", "unknown")
        style = attributes.get("style", "unknown")
        formality = attributes.get("formality_score", "unknown")
        pattern = attributes.get("pattern", "unknown")

        return (
            f"ID: {item_id} - {color} {pattern} item, {style} style, formality: {formality}/10.\n"
            f"Description: {description}"
        )

    contents = [
        f"You are a fashion assistant. The user wants an outfit recommendation for a **{context}** occasion.",
        "Choose ONE shirt, ONE pants, and ONE outerwear item from the items below. Each is shown with an image and description.",
        "Return the outfit like this:\n"
        "OUTFIT:\n- shirt_id: [ID]\n- pants_id: [ID]\n- outerwear_id: [ID]\nREASONING: [Why you chose them together]"
    ]

    def add_items_to_contents(label, items):
        contents.append(f"### {label.upper()} ###")
        for item in items:
            contents.append(pil_to_inline_data(item["image"]))
            contents.append(format_item_text(item))

    add_items_to_contents("shirts", shirts)
    add_items_to_contents("pants", pants)
    add_items_to_contents("outerwear", outerwear)


    response = client.models.generate_content(
        model="gemini-1.5-pro",
        contents=contents,
        config=types.GenerateContentConfig(response_modalities=["Text"])
    )

    response_text = response.text

    import re
    shirt_match = re.search(r'shirt_id:\s*(\w+)', response_text)
    pants_match = re.search(r'pants_id:\s*(\w+)', response_text)
    outerwear_match = re.search(r'outerwear_id:\s*(\w+)', response_text)
    reasoning_match = re.search(r'REASONING:\s*(.*?)($|\n\n)', response_text, re.DOTALL)

    return {
        "outfit": {
            "shirt_id": shirt_match.group(1) if shirt_match else None,
            "pants_id": pants_match.group(1) if pants_match else None,
            "outerwear_id": outerwear_match.group(1) if outerwear_match else None
        },
        "reasoning": reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided",
        "raw_response": response_text
    }


SyntaxError: invalid syntax (2830201818.py, line 1)